# LangChain: Embedding

### Install package

In [1]:
!pip install -q --upgrade langchain langchain-community langchain-openai langchain-core chromadb faiss-cpu
# 安裝langchain + google or openai
!pip install -q "langchain[google-genai]" "langchain[openai]" tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 8.9 MB/s eta 0:00:0

### 設定

In [2]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import (
    JsonOutputParser,
    CommaSeparatedListOutputParser,
)
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import (
    PromptTemplate, # 提示詞模板
    ChatPromptTemplate,
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_classic.chains import (
    LLMChain,
    SimpleSequentialChain,
    RouterChain,
)
# 用於在對話中運行自定義邏輯。例如，你可以使用它來處理用戶輸入前的一些預處理操作，或者在生成回應後進行一些後處理操作。
from langchain_core.runnables import RunnableLambda
from google.colab import userdata
from rich.pretty import pprint
import os

# 設定API Key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
# 建立模型
model = init_chat_model("google_genai:gemini-2.5-flash-lite")

## **3. Text embedding models**

Embeddings 類別 (Class) 是設計用於與文本嵌入模型交互，為所有嵌入模型提供者（OpenAI、Cohere、Hugging Face 等）提供標準介面。

Embeddings 創建一段文本的向量表示。這意味著我們可以在向量空間中表示文本，並執行語義搜尋之類的操作，在向量空間中找到最相似的文本片段。

LangChain 中的 Embeddings 類別的基底提供了兩種方法：一種用於嵌入文件，另一種用於嵌入查詢 (query)。 前者採用多個文本 (text) 作為輸入，而後者僅可輸入單一文本 (text)。 將它們作為兩種不同方法是因為某些嵌入提供者對兩者有不同的嵌入方法。

![Introducing text and code embeddings](<https://cdn.openai.com/embeddings/draft-20220124e/vectors-mobile-1.svg> 'Introducing text and code embeddings')

source：[Introducing text and code embeddings](<https://openai.com/blog/introducing-text-and-code-embeddings> 'Introducing text and code embeddings')

**API Reference: [OpenAIEmbeddings](<https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html> 'OpenAIEmbeddings')**

In [3]:
from langchain_openai import OpenAIEmbeddings
# 利用OpenAI服務建立embedding model
OPENAI_KEY = userdata.get("OPENAI_API_KEY")
embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

### **3.1 embed_documents**

嵌入文字列表
使用 `.embed_documents` 嵌入字串清單。

In [4]:
# 將字串輸入embedding model取得embeddings
embedded_doc = embeddings_model.embed_documents(
    [
        'Hi there!',
        'Oh, hello!',
        'What\'s your name?',
        'My friends call me World',
        'Hello World!'
    ]
)

In [5]:
for embedding in embedded_doc:
    print(len(embedding)) # embedding 長度

1536
1536
1536
1536
1536


In [6]:
# 'Hi there!' embedding
embedded_doc[0]

[-0.020325319841504097,
 -0.007096723187714815,
 -0.022839006036520004,
 -0.026279456913471222,
 -0.037527572363615036,
 0.02163294516503811,
 -0.006144568789750338,
 -0.008975640870630741,
 0.008524954319000244,
 -0.016618264839053154,
 0.02683805488049984,
 -0.007356978487223387,
 -0.013545980677008629,
 -0.024133935570716858,
 0.006512735038995743,
 -0.020198365673422813,
 0.02426088973879814,
 -0.014739347621798515,
 0.016427835449576378,
 -0.01647861674427986,
 -0.007204633671790361,
 -0.008080615662038326,
 0.004694120492786169,
 -0.002066174754872918,
 -0.014802824705839157,
 -0.005989050026983023,
 -0.0020868047140538692,
 -0.02301674149930477,
 0.019855590537190437,
 -0.031535349786281586,
 0.012860430404543877,
 0.011622629128396511,
 -0.008518606424331665,
 -0.009477108716964722,
 -0.001813853858038783,
 -0.027422042563557625,
 -0.008264699019491673,
 0.002078870078548789,
 0.024006983265280724,
 -0.008734428323805332,
 0.02349916659295559,
 0.0009116876753978431,
 0.0097691

### **3.2 embed_query**

嵌入單一查詢
使用“.embed_query”嵌入單一文字片段（例如，為了與其他嵌入的文字片段進行比較）。

In [7]:
embedded_query = embeddings_model.embed_query('Hi there~~~~')
print("embedding長度", len(embedded_query))

embedding長度 1536


## **4. Vector stores**

儲存和搜尋非結構化資料的最常見方法之一是嵌入它並儲存產生的嵌入向量，然後在查詢時嵌入非結構化文本 (稱為 query) 並檢索與 query「最相似」的嵌入向量。 向量儲存 (Vector stores) 負責儲存嵌入資料並為執行向量搜尋。

source：[Vector stores](<https://python.langchain.com/docs/modules/data_connection/vectorstores/> 'Vector stores')

以下範例為向量儲存 (Vector stores) 相關的基本功能。 使用向量儲存 (Vector stores) 的關鍵部分是建立要放入其中的向量，這通常是透過嵌入來創建的。 因此，建議您在深入研究之前熟悉 [文本嵌入模型](<https://python.langchain.com/v0.2/docs/how_to/embed_text/> 'embedding model interfaces')。

有許多不錯的的向量儲存 (Vector stores) 選擇，以下是一些免費、開源且完全在本地上運行的選擇：
- Chroma
- FAISS
- Lance

下面以 **Chroma** 和 **FAISS** 作為範例。

### **4.1 chromadb**

使用 `chroma` 向量資料庫，它作為 library 在本機電腦上運行。

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters  import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import Chroma
from chromadb.errors import InvalidDimensionException

In [9]:
# 讀取文件使用splitter切割成chunks
raw_documents = TextLoader('state_of_the_union.txt').load()

text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
    add_start_index = True,
)

documents = text_splitter.split_documents(raw_documents)

In [13]:
pprint(documents), print(len(documents))

[
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 0},
│   │   page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 493},
│   │   page_content='Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 942},
│   │   page_content='Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. \n\nIn this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight. \n\nLet each of us here tonight in this Chamber send an unmistakable signal to Ukraine and to the world.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 1369},
│   │   page_content='Please rise if you are able and show that, Yes, we the United States of America stand with the Ukrainian people. \n\nThroughout our history we’ve learned this lesson when dictators do not pay a price for their aggression they cause more chaos.   \n\nThey keep moving.   \n\nAnd the costs and the threats to America and the world keep rising.   \n\nThat’s why the NATO Alliance was created to secure peace and stability in Europe after World War 2.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 1811},
│   │   page_content='The United States is a member along with 29 other nations. \n\nIt matters. American diplomacy matters. American resolve matters. \n\nPutin’s latest attack on Ukraine was premeditated and unprovoked. \n\nHe rejected repeated efforts at diplomacy. \n\nHe thought the West and NATO wouldn’t respond. And he thought he could divide us at home. Putin was wrong. We were ready.  Here is what we did.   \n\nWe prepared extensively and carefully.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 2201},
│   │   page_content='We prepared extensively and carefully. \n\nWe spent months building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. \n\nI spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  \n\nWe countered Russia’s lies with truth.   \n\nAnd now that he has acted the free world is holding him accountable.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 2685},
│   │   page_content='Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. \n\nWe are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. \n\nTogether with our allies –we are right now enforcing powerful econ

90


(None, None)

In [14]:
# 利用OpenAI服務建立embedding model
embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
# 將每個chunks轉換成embeddings儲存至chroma db
db = Chroma.from_documents(documents, embedding=embeddings_model)

**API Reference: [Chroma](<https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html> 'Chroma')**

#### **4.1.1 Similarity search**

In [15]:
query = 'What did the president say about Ketanji Brown Jackson'
# 相似度搜尋
docs = db.similarity_search(query)
pprint(docs)
# 最接近的chunk內容
print(docs[0].page_content)

[
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 29486},
│   │   page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'
│   ),
│   Document(
│   │   metadata={'start_index': 31766, 'source': 'state_of_the_union.txt'},
│   │   page_content='As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 29680},
│   │   page_content='A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 19609},
│   │   page_content='But in my administration, the watchdogs have been welcomed back. \n\nWe’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  \n\nAnd tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. \n\nBy the end of this year, the deficit will be down to less than half what it was before I took office.  \n\nThe only president ever to cut the deficit by more than one trillion dollars in a single year.'
│   )
]

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### **4.2 FAISS**

使用 `FAISS` 向量資料庫，利用 Facebook AI 相似性搜尋 (Facebook AI Similarity Search, FAISS) 函式庫。

In [18]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters  import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import FAISS

In [19]:
# 讀取文件使用splitter切割成chunks
raw_documents = TextLoader('state_of_the_union.txt').load()

text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
    length_function = len,
    add_start_index = True,
)

documents = text_splitter.split_documents(raw_documents)

In [20]:
# 將每個chunks轉換成embeddings儲存至FAISS db
db = FAISS.from_documents(documents, embedding=embeddings_model)

**API Reference: [FAISS](<https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html> 'FAISS')**

#### **4.2.1 Similarity search**

In [21]:
query = 'What did the president say about Ketanji Brown Jackson'
docs = db.similarity_search(query)
pprint(docs)

[
│   Document(
│   │   id='657892f1-c08e-4025-90c1-0f315782f5c0',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 29486},
│   │   page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'
│   ),
│   Document(
│   │   id='f20b1a52-5aa1-4a40-b024-259acdf126c2',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 31766},
│   │   page_content='As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.'
│   ),
│   Document(
│   │   id='9eab0fe5-4bc3-4920-8c11-96fcec08d4d2',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 29680},
│   │   page_content='A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system.'
│   ),
│   Document(
│   │   id='a610906b-b058-4ec4-9b29-628ef5388c2e',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 19609},
│   │   page_content='But in my administration, the watchdogs have been welcomed back. \n\nWe’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  \n\nAnd tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. \n\nBy the end of this year, the deficit will be down to less than half what it was before I took office.  \n\nThe only president ever to cut the deficit by more than one trillion dollars in a single year.'
│   )
]

### **Practice**

請使用資料做成 Document 後，放入 Vector Store 當中，並嘗試查詢與問題最相關的向量。

In [ ]:
chroma_db = Chroma.from_documents(documents, embedding=embeddings_model)
faiss_db = FAISS.from_documents(documents, embedding=embeddings_model)

query = "TODO"

docs_chroma = chroma_db.similarity_search(query)
docs_faiss = faiss_db.similarity_search(query)

pprint(docs_chroma)
pprint(docs_faiss)

[
│   Document(
│   │   metadata={'start_index': 3579, 'source': 'state_of_the_union.txt'},
│   │   page_content='The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.  \n\nWe are joining with our European allies to find and seize your yachts your luxury apartments your private jets. We are coming for your ill-begotten gains.'
│   ),
│   Document(
│   │   metadata={'start_index': 3579, 'source': 'state_of_the_union.txt'},
│   │   page_content='The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.  \n\nWe are joining with our European allies to find and seize your yachts your luxury apartments your private jets. We are coming for your ill-begotten gains.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 36469},
│   │   page_content='More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy.'
│   ),
│   Document(
│   │   metadata={'start_index': 36469, 'source': 'state_of_the_union.txt'},
│   │   page_content='More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy.'
│   )
]

[
│   Document(
│   │   id='46da09fc-692c-45a5-8943-add7040bccb6',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 3579},
│   │   page_content='The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.  \n\nWe are joining with our European allies to find and seize your yachts your luxury apartments your private jets. We are coming for your ill-begotten gains.'
│   ),
│   Document(
│   │   id='7459ec71-eab2-4ace-b3d1-1fa901da27f4',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 36469},
│   │   page_content='More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy.'
│   ),
│   Document(
│   │   id='9a1ada20-70b5-4daf-b6ec-1c71d19fb9f9',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 38343},
│   │   page_content='Now is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.'
│   ),
│   Document(
│   │   id='238f6eba-e979-4c8b-8b33-07a7c4ba7235',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 30598},
│   │   page_content='We can do all this while keeping lit the torch of liberty that has led generations of immigrants to this land—my forefathers and so many of yours. \n\nProvide a pathway to citizenship for Dreamers, those on temporary status, farm workers, and essential workers. \n\nRevise our laws so businesses have the workers they need and families don’t wait decades to reunite. \n\nIt’s not only the right thing to do—it’s the economically smart thing to do.'
│   )
]

## **5. Retrievers**

檢索器 (Retrievers) 是一個接口，它根據非結構化的 query 返回文件。 它比向量存儲 (Vector store) 更通用。 檢索器 (Retrievers) 不需要能夠儲存文件，只需返回（或檢索）它們即可。 向量儲存 (Vector store) 可以用作檢索器 (Retrievers) 的骨幹，但也有其他類型的檢索器 (Retrievers)。
檢索器 (Retrievers) 實作 Runnable 接口，這是 [LangChain 表達式語言 LangChain Expression Language (LCEL)](<https://python.langchain.com/docs/expression_language/> 'LangChain Expression Language (LCEL)') 的基本構建塊 (block)。 這意味著它們支援 `invoke`、`ainvoke`、`stream`、`astream`、`batch`、`abatch`、`astream_log` 的調用方法。

檢索器接受字串查詢作為輸入，並傳回 `Documents` 清單作為輸出。

在此範例中，我們將使用支援 Chroma 的檢索器 (Retrievers)。

In [22]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters  import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

In [23]:
raw_documents = TextLoader('state_of_the_union.txt').load()

text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
    length_function = len,
    add_start_index = True,
)

documents = text_splitter.split_documents(raw_documents)
pprint(documents[:3])

[
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 0},
│   │   page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 493},
│   │   page_content='Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 942},
│   │   page_content='Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. \n\nIn this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight. \n\nLet each of us here tonight in this Chamber send an unmistakable signal to Ukraine and to the world.'
│   )
]

In [24]:
# 建立chroma faiss 資料庫
Chroma().delete_collection()
chroma_db = Chroma.from_documents(documents, embedding=embeddings_model)

faiss_db = FAISS.from_documents(documents, embedding=embeddings_model)

/tmp/ipython-input-1894817577.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  Chroma().delete_collection()


### **5.1 Simple Retriever**

Simple Retriever 是 Langchain 提供的一個基本檢索器 (Retrievers)，用於查詢向量儲存 (Vector store)。

source：[Deep Dive into the Internals of Langchain Vector Store Retriever](<https://rito.hashnode.dev/deep-dive-into-the-internals-of-langchain-vector-store-retriever> 'Deep Dive into the Internals of Langchain Vector Store Retriever'):

這裡我們建立了一個 FAISS DB 檢索器，其中 `k` 值為 2。`k factor` 決定檢索器應傳回多少文件。可以使用 `get_relevant_documents` 檢索文件。以下是檢索到的兩份文件：

In [32]:
simple_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7973155da1e0>, search_kwargs={'k': 2})

In [25]:
# 取得Chroma檢索器
simple_retriever = chroma_db.as_retriever(search_kwargs={'k': 2})
question = 'What did the president say about technology?'
result = simple_retriever.invoke(question)

pprint(result)

[
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 12096},
│   │   page_content='That’s why it is so important to pass the Bipartisan Innovation Act sitting in Congress that will make record investments in emerging technologies and American manufacturing. \n\nLet me give you one example of why it’s so important to pass it. \n\nIf you travel 20 miles east of Columbus, Ohio, you’ll find 1,000 empty acres of land. \n\nIt won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built.'
│   ),
│   Document(
│   │   metadata={'start_index': 10383, 'source': 'state_of_the_union.txt'},
│   │   page_content='We’re going to have an infrastructure decade. \n\nIt is going to transform America and put us on a path to win the economic competition of the 21st Century that we face with the rest of the world—particularly with China.  \n\nAs I’ve told Xi Jinping, it is never a good bet to bet against the American people. \n\nWe’ll create good jobs for millions of Americans, modernizing roads, airports, ports, and waterways all across America.'
│   )
]

In [26]:
# 取得FAISS檢索器
simple_retriever = faiss_db.as_retriever(search_kwargs={'k': 2})
question = 'What did the president say about technology?'
result = simple_retriever.invoke(question)

pprint(result)

[
│   Document(
│   │   id='26d863f6-da70-4032-aa65-03e6f783c8cb',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 12096},
│   │   page_content='That’s why it is so important to pass the Bipartisan Innovation Act sitting in Congress that will make record investments in emerging technologies and American manufacturing. \n\nLet me give you one example of why it’s so important to pass it. \n\nIf you travel 20 miles east of Columbus, Ohio, you’ll find 1,000 empty acres of land. \n\nIt won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built.'
│   ),
│   Document(
│   │   id='61e6df93-63ed-4395-b92e-23e24e9d556b',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 10383},
│   │   page_content='We’re going to have an infrastructure decade. \n\nIt is going to transform America and put us on a path to win the economic competition of the 21st Century that we face with the rest of the world—particularly with China.  \n\nAs I’ve told Xi Jinping, it is never a good bet to bet against the American people. \n\nWe’ll create good jobs for millions of Americans, modernizing roads, airports, ports, and waterways all across America.'
│   )
]

除了上述的方式，LangChain 中提供了不同的檢索器 (Retriever)：
- [MultiQueryRetriever](<https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever> 'MultiQueryRetriever')
- [Contextual compression](<https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression/> 'Contextual compression')
- [Ensemble Retriever](<https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble> 'Ensemble Retriever')
- [MultiVector Retriever](<https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector> 'MultiVector Retriever')
- [Parent Document Retriever](<https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever> 'Parent Document Retriever')
- [Self-querying](<https://python.langchain.com/docs/modules/data_connection/retrievers/self_query> 'Self-querying')
- [Time-weighted vector store retriever](<https://python.langchain.com/docs/modules/data_connection/retrievers/time_weighted_vectorstore> 'Vector store-backed retriever')
- [Vector store-backed retriever](<https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore> '')
- [WebResearchRetriever](<https://python.langchain.com/docs/modules/data_connection/retrievers/web_research> 'WebResearchRetriever')

下面範例將以 `MultiQueryRetriever` 和 `Contextual compression` 為範例實作。

### **5.2 MultiQuery Retriever**

在這裡，我們首先使用 LLM 根據使用者問題的不同觀點來建立各種問題。 對於每個問題，我們都會從向量儲存 (Vector store) 找到相關文件。 然後，我們檢查所有文件中的重複項，以製作獨一無二文件的特殊清單。

![MultiQuery Retriever](<https://cdn.hashnode.com/res/hashnode/image/upload/v1692507354003/b7d252b1-ae95-4051-b0f5-a3cb048d260c.png?auto=compress,format&format=webp> 'MultiQuery Retriever')

source：[Deep Dive into the Internals of Langchain Vector Store Retriever](<https://rito.hashnode.dev/deep-dive-into-the-internals-of-langchain-vector-store-retriever> 'Deep Dive into the Internals of Langchain Vector Store Retriever'):

In [27]:
from langchain_openai import ChatOpenAI
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
import logging

In [28]:
question = 'What did the president say about technology?'

In [29]:
# MultiQueryRetriever：可以自動幫一個問題，產生多個不同角度的查詢，提高找資料的命中率
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=faiss_db.as_retriever(),  # 使用 FAISS 資料庫作為基礎的 retriever（檢索器）
    llm=model                           # 指定用哪個 LLM 來產生多樣化的查詢問題
)

# 啟動 Python logging（記錄程式輸出）
logging.basicConfig()

# 將 langchain.retrievers.multi_query 這個模組的 log 等級設成 INFO
# 這樣可以在執行時看到 MultiQueryRetriever 做了哪些「多查詢產生」的細節
logging.getLogger('langchain_classic.retrievers.multi_query').setLevel(logging.INFO)

**API Reference: [MultiQueryRetriever](<https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html> 'MultiQueryRetriever')**

In [30]:
multi_query_result = multi_query_retriever.invoke(input=question)

pprint(multi_query_result)

INFO:langchain_classic.retrievers.multi_query:Generated queries: ["What were the president's recent remarks on technological advancements?", "Can you provide a summary of the president's statements regarding technology policy?", 'What are the key points the president has made about the role of technology in society?']


[
│   Document(
│   │   id='26d863f6-da70-4032-aa65-03e6f783c8cb',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 12096},
│   │   page_content='That’s why it is so important to pass the Bipartisan Innovation Act sitting in Congress that will make record investments in emerging technologies and American manufacturing. \n\nLet me give you one example of why it’s so important to pass it. \n\nIf you travel 20 miles east of Columbus, Ohio, you’ll find 1,000 empty acres of land. \n\nIt won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built.'
│   ),
│   Document(
│   │   id='bf947b08-9ea0-40d4-be5d-58bef1088c33',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 31766},
│   │   page_content='As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.'
│   ),
│   Document(
│   │   id='31119bb8-36af-4030-93b3-105982f4c5bd',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 37853},
│   │   page_content='We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. \n\nThe only nation that can be defined by a single word: possibilities. \n\nSo on this night, in our 245th year as a nation, I have come to report on the State of the Union. \n\nAnd my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today.'
│   ),
│   Document(
│   │   id='61e6df93-63ed-4395-b92e-23e24e9d556b',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 10383},
│   │   page_content='We’re going to have an infrastructure decade. \n\nIt is going to transform America and put us on a path to win the economic competition of the 21st Century that we face with the rest of the world—particularly with China.  \n\nAs I’ve told Xi Jinping, it is never a good bet to bet against the American people. \n\nWe’ll create good jobs for millions of Americans, modernizing roads, airports, ports, and waterways all across America.'
│   ),
│   Document(
│   │   id='7d611754-3a21-424d-bf43-3e2603810b3b',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 36469},
│   │   page_content='More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy.'
│   ),
│   Document(
│   │   id='769830cc-2ad7-419e-ad5c-8653e995fde3',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 15684},
│   │   page_content='I call it building a better America. \n\nMy plan to fight inflation will lower your costs and lower the deficit. \n\n17 Nobel laureates in economics say my plan will ease long-term inflationary pressures. Top business leaders and most Americans support my plan. And here’s the plan: \n\nFirst – cut the cost of prescription drugs. Just look at insulin. One in ten Americans has diabetes. In Virginia, I met a 13-year-old boy named Joshua Davis.'
│   ),
│   Document(
│   │   id='0f4fbba5-b1f1-4378-86bd-ad6a1c8989ef',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 10813},
│   │   page_content='And we’ll do

當我們運行檢索器 (Retrievers) 時，會根據給定的查詢產生 3 個 query：
1. Can you provide any information on the president's views regarding technology?
2. I'm interested in knowing the president's statements or opinions on technology. Could you share any details?
3. Could you please share any insights or remarks made by the president about technology?

取得所有查詢的相關文件並將其傳回給使用者刪除重複。 以下是上述 3 個查詢傳回的文件。

In [31]:
pprint(multi_query_result)

[
│   Document(
│   │   id='26d863f6-da70-4032-aa65-03e6f783c8cb',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 12096},
│   │   page_content='That’s why it is so important to pass the Bipartisan Innovation Act sitting in Congress that will make record investments in emerging technologies and American manufacturing. \n\nLet me give you one example of why it’s so important to pass it. \n\nIf you travel 20 miles east of Columbus, Ohio, you’ll find 1,000 empty acres of land. \n\nIt won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built.'
│   ),
│   Document(
│   │   id='bf947b08-9ea0-40d4-be5d-58bef1088c33',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 31766},
│   │   page_content='As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.'
│   ),
│   Document(
│   │   id='31119bb8-36af-4030-93b3-105982f4c5bd',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 37853},
│   │   page_content='We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. \n\nThe only nation that can be defined by a single word: possibilities. \n\nSo on this night, in our 245th year as a nation, I have come to report on the State of the Union. \n\nAnd my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today.'
│   ),
│   Document(
│   │   id='61e6df93-63ed-4395-b92e-23e24e9d556b',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 10383},
│   │   page_content='We’re going to have an infrastructure decade. \n\nIt is going to transform America and put us on a path to win the economic competition of the 21st Century that we face with the rest of the world—particularly with China.  \n\nAs I’ve told Xi Jinping, it is never a good bet to bet against the American people. \n\nWe’ll create good jobs for millions of Americans, modernizing roads, airports, ports, and waterways all across America.'
│   ),
│   Document(
│   │   id='7d611754-3a21-424d-bf43-3e2603810b3b',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 36469},
│   │   page_content='More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy.'
│   ),
│   Document(
│   │   id='769830cc-2ad7-419e-ad5c-8653e995fde3',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 15684},
│   │   page_content='I call it building a better America. \n\nMy plan to fight inflation will lower your costs and lower the deficit. \n\n17 Nobel laureates in economics say my plan will ease long-term inflationary pressures. Top business leaders and most Americans support my plan. And here’s the plan: \n\nFirst – cut the cost of prescription drugs. Just look at insulin. One in ten Americans has diabetes. In Virginia, I met a 13-year-old boy named Joshua Davis.'
│   ),
│   Document(
│   │   id='0f4fbba5-b1f1-4378-86bd-ad6a1c8989ef',
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 10813},
│   │   page_content='And we’ll do

### **5.3 ContextualCompression Retriever**

在 `ContextualCompressionRetriever` 中，使用給定 query 的上下文對從向量儲存 (Vector store) 中檢索到的文件進行壓縮，並從檢索到的文件中過濾掉不相關的內容。

![ContextualCompression Retriever](<https://cdn.hashnode.com/res/hashnode/image/upload/v1692508115445/9abc2227-7c8f-46f0-9f1c-e5606c33e932.png?auto=compress,format&format=webp> 'ContextualCompression Retriever')

source：[Deep Dive into the Internals of Langchain Vector Store Retriever](<https://rito.hashnode.dev/deep-dive-into-the-internals-of-langchain-vector-store-retriever> 'Deep Dive into the Internals of Langchain Vector Store Retriever'):

ContextualCompressionRetriever 是一個「壓縮型檢索器」，
它可以把原本「撈回來的長文資料」，自動濃縮成只保留跟問題有關的內容。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

In [ ]:
question = 'What did the president say about technology?'

In [ ]:
# 壓縮型檢索器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=LLMChainExtractor.from_llm(model),
    base_retriever=faiss_db.as_retriever()
)

In [ ]:
# 輸入問題檢索
compressed_docs = compression_retriever.invoke(question)
pprint(compressed_docs)

[
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 12096},
│   │   page_content='That’s why it is so important to pass the Bipartisan Innovation Act sitting in Congress that will make record investments in emerging technologies and American manufacturing.'
│   ),
│   Document(
│   │   metadata={'source': 'state_of_the_union.txt', 'start_index': 12573},
│   │   page_content='Smartphones. The Internet. Technology we have yet to invent.'
│   )
]